## Homework 3 and 4 - Applications Using MRJob

In [1]:
# general imports
import os
import re
import sys
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# tell matplotlib not to open a new window
%matplotlib inline

# automatically reload modules 
%reload_ext autoreload
%autoreload 2

In [2]:
# print some configuration details for future replicability.
print 'Python Version: %s' % (sys.version.split('|')[0])
hdfs_conf = !hdfs getconf -confKey fs.defaultFS ### UNCOMMENT ON DOCKER
#hdfs_conf = !hdfs getconf -confKey fs.default.name ### UNCOMMENT ON ALTISCALE
print 'HDFS filesystem running at: \n\t %s' % (hdfs_conf[0])

Python Version: 2.7.14 
HDFS filesystem running at: 
	 hdfs://quickstart.cloudera:8020


In [3]:
JAR_FILE = "/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.7.0.jar"
HDFS_DIR = "/user/root/HW3"
HOME_DIR = "/media/notebooks/SP18-1-maynard242" # FILL IN HERE eg. /media/notebooks/w261-main/Assignments
# save path for use in Hadoop jobs (-cmdenv PATH={PATH})
from os import environ
PATH  = environ['PATH']

In [5]:
#!hdfs dfs -mkdir HW3
!hdfs dfs -ls

In [6]:
%%writefile example1.txt
Unix,30
Solaris,10
Linux,25
Linux,20
HPUX,100
AIX,25

Overwriting example1.txt


In [7]:
%%writefile example2.txt
foo foo quux labs foo bar jimi quux jimi jimi
foo  jimi jimi
data mining is data science


Overwriting example2.txt


In [8]:
%%writefile WordCount.py

from mrjob.job import MRJob
import re
 
WORD_RE = re.compile(r"[\w']+")
 
class MRWordFreqCount(MRJob):
    def mapper(self, _, line):
        for word in WORD_RE.findall(line):
            yield word.lower(), 1
     
    def combiner(self, word, counts):
        yield word, sum(counts)

    #hello, (1,1,1,1,1,1): using a combiner? NO and YEs
    def reducer(self, word, counts):
        yield word, sum(counts)

if __name__ == '__main__':
    MRWordFreqCount.run()

Overwriting WordCount.py


In [9]:
!python WordCount.py -r hadoop --cmdenv PATH=/opt/anaconda/bin:$PATH example2.txt

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in $PATH...
Found hadoop binary: /usr/bin/hadoop
Using Hadoop version 2.6.0
Looking for Hadoop streaming jar in /home/hadoop/contrib...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Creating temp directory /tmp/WordCount.root.20180219.120016.451572
Copying local files to hdfs:///user/root/tmp/mrjob/WordCount.root.20180219.120016.451572/files/...
Running step 1 of 1...
  packageJobJar: [] [/usr/jars/hadoop-streaming-2.6.0-cdh5.7.0.jar] /tmp/streamjob8325369025906361369.jar tmpDir=null
  Connecting to ResourceManager at /0.0.0.0:8032
  Connecting to ResourceManager at /0.0.0.0:8032
  Total input paths to process : 1
  number of splits:2
  Submitting tokens for job: job_1519041572493_0001
  Submitted application application_1519041572493_0001
  The url to track the job: http://doc

In [10]:
from WordCount import MRWordFreqCount
mr_job = MRWordFreqCount(args=['example2.txt'])
with mr_job.make_runner() as runner: 
    runner.run()
    # stream_output: get access of the output 
    for line in runner.stream_output():
        print mr_job.parse_output_line(line)

(u'jimi', 5)
(u'quux', 2)
(u'science', 1)
(u'foo', 4)
(u'bar', 1)
(u'data', 2)
(u'is', 1)
(u'labs', 1)
(u'mining', 1)


In [22]:
%%writefile AnotherWordCount.py

from mrjob.job import MRJob

class MRAnotherWordCount(MRJob):
    
    def mapper (self,_,line):
        yield "chars", len(line)
        yield "words", len(line.split())
        yield 'lines', 1
        
    def reducer (self, key, values):
        yield key, sum(values)
        
if __name__ == '__main__':
    MRAnotherWordCount.run()

Overwriting AnotherWordCount.py


In [26]:
!python AnotherWordCount.py example2.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/AnotherWordCount.root.20180203.141743.462604
Streaming final output from /tmp/AnotherWordCount.root.20180203.141743.462604/output...
"words"	18
"chars"	86
"lines"	3
Removing temp directory /tmp/AnotherWordCount.root.20180203.141743.462604...


In [5]:
%%writefile AnotherWC3.py
# Copyright 2009-2010 Yelp
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""An implementation of wc as an MRJob.
This is meant as an example of why mapper_final is useful."""
from mrjob.job import MRJob


class MRWordCountUtility(MRJob):

    def __init__(self, *args, **kwargs):
        super(MRWordCountUtility, self).__init__(*args, **kwargs)
        self.chars = 0
        self.words = 0
        self.lines = 0

    def mapper(self, _, line):
        # Don't actually yield anything for each line. Instead, collect them
        # and yield the sums when all lines have been processed. The results
        # will be collected by the reducer.
        self.chars += len(line) + 1  # +1 for newline
        self.words += sum(1 for word in line.split() if word.strip())
        self.lines += 1

    def mapper_final(self):
        yield('chars', self.chars)
        yield('words', self.words)
        yield('lines', self.lines)

    def reducer(self, key, values):
        yield(key, sum(values))


if __name__ == '__main__':
    MRWordCountUtility.run()

Writing AnotherWC3.py


In [8]:
!python AnotherWC3.py -r hadoop --cmdenv PATH=/opt/anaconda/bin:$PATH example2.txt

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in $PATH...
Found hadoop binary: /usr/bin/hadoop
Using Hadoop version 2.6.0
Looking for Hadoop streaming jar in /home/hadoop/contrib...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Creating temp directory /tmp/AnotherWC3.root.20180210.114605.598768
Copying local files to hdfs:///user/root/tmp/mrjob/AnotherWC3.root.20180210.114605.598768/files/...
Running step 1 of 1...
  packageJobJar: [] [/usr/jars/hadoop-streaming-2.6.0-cdh5.7.0.jar] /tmp/streamjob6426766568329965202.jar tmpDir=null
  Connecting to ResourceManager at /0.0.0.0:8032
  Connecting to ResourceManager at /0.0.0.0:8032
  Total input paths to process : 1
  number of splits:2
  Submitting tokens for job: job_1518262877936_0001
  Submitted application application_1518262877936_0001
  The url to track the job: http://d

In [43]:
%%writefile AnotherWC2.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re

WORD_RE = re.compile(r"[\w']+")


class MRMostUsedWord(MRJob):

    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_words,
                   combiner=self.combiner_count_words,
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_find_max_word)
        ]

    def mapper_get_words(self, _, line):
        # yield each word in the line
        for word in WORD_RE.findall(line):
            self.increment_counter('group', 'counter_name', 1)
            yield (word.lower(), 1)

    def combiner_count_words(self, word, counts):
        # optimization: sum the words we've seen so far
        yield (word, sum(counts))

    def reducer_count_words(self, word, counts):
        # send all (num_occurrences, word) pairs to the same reducer.
        # num_occurrences is so we can easily use Python's max() function.
        yield None, (sum(counts), word)

    # discard the key; it is just None
    def reducer_find_max_word(self, _, word_count_pairs):
        # each item of word_count_pairs is (count, word),
        # so yielding one results in key=counts, value=word
        yield max(word_count_pairs)


if __name__ == '__main__':
    MRMostUsedWord.run()

Overwriting AnotherWC2.py


In [49]:
!python WordCount.py example2.txt --output-dir mrJobOutput

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/WordCount.root.20180203.150000.135247
Streaming final output from mrJobOutput...
"labs"	1
"jimi"	5
"foo"	4
"science"	1
"quux"	2
"bar"	1
"mining"	1
"is"	1
"data"	2
Removing temp directory /tmp/WordCount.root.20180203.150000.135247...


In [68]:
!ls -las mrJobOutput/
!cat mrJobOutput/part-0000*

total 44
4 drwxr-xr-x 2 root root 4096 Feb  3 14:52 .
4 drwxrwxr-x 4 1000 1000 4096 Feb  3 15:16 ..
4 -rw-r--r-- 1 root root    8 Feb  3 15:22 part-00000
4 -rw-r--r-- 1 root root    9 Feb  3 15:22 part-00001
4 -rw-r--r-- 1 root root    8 Feb  3 15:22 part-00002
4 -rw-r--r-- 1 root root    7 Feb  3 15:22 part-00003
4 -rw-r--r-- 1 root root    9 Feb  3 15:22 part-00004
4 -rw-r--r-- 1 root root    9 Feb  3 15:22 part-00005
4 -rw-r--r-- 1 root root   11 Feb  3 15:22 part-00006
4 -rw-r--r-- 1 root root    9 Feb  3 15:22 part-00007
4 -rw-r--r-- 1 root root   12 Feb  3 15:22 part-00008
0 -rw-r--r-- 1 root root    0 Feb  3 15:22 part-00009
"bar"	1
"data"	2
"foo"	4
"is"	1
"jimi"	5
"labs"	1
"mining"	1
"quux"	2
"science"	1


In [72]:
%%writefile WordCount2.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re
 
WORD_RE = re.compile(r"[\w']+")
 
class MRWordFreqCount(MRJob):
    SORT_VALUES = True
    def mapper(self, _, line):
        for word in WORD_RE.findall(line):
            self.increment_counter('group', 'mapper', 1)
            yield word.lower(), 1
            
    def jobconfqqqq(self):  #assume we had second job to sort the word counts in decreasing order of counts
        orig_jobconf = super(MRWordFreqCount, self).jobconf()        
    'mapred.reduce.tasks': '1',
        }
        combined_jobconf = orig_jobconf
        combined_jobconf.update(custom_jobconf)
        self.jobconf = combined_jobconf
        return combined_jobconf      custom_jobconf = {  #key value pairs
            'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapred.text.key.comparator.options': '-k2,2nr',
          


    def combiner(self, word, counts):
        self.increment_counter('group', 'combiner', 1)
        yield word, sum(counts)

    def reducer(self, word, counts):
        self.increment_counter('group', 'reducer', 1)
        yield word, sum(counts)

    def steps(self):
        return [MRStep(
                mapper = self.mapper, 
                combiner = self.combiner,
                reducer = self.reducer,
                #,
#                jobconf = self.jobconfqqqq
 
#            jobconf = {'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
#                       'mapred.text.key.comparator.options':'-k1r',
#                       'mapred.reduce.tasks' : 1}   
       
        
            )]
     


if __name__ == '__main__':
    MRWordFreqCount.run()

Overwriting WordCount2.py


In [73]:
!python WordCount2.py --jobconf numReduceTasks=1 example2.txt --output-dir mrJobOutput

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/WordCount2.root.20180203.152437.411298

Counters: 3
	group
		combiner=11
		mapper=18
		reducer=9

Streaming final output from mrJobOutput...
"labs"	1
"jimi"	5
"foo"	4
"science"	1
"quux"	2
"bar"	1
"mining"	1
"is"	1
"data"	2
Removing temp directory /tmp/WordCount2.root.20180203.152437.411298...


In [71]:
!ls -las mrJobOutput/

total 44
4 drwxr-xr-x 2 root root 4096 Feb  3 14:52 .
4 drwxrwxr-x 4 1000 1000 4096 Feb  3 15:22 ..
4 -rw-r--r-- 1 root root    8 Feb  3 15:23 part-00000
4 -rw-r--r-- 1 root root    9 Feb  3 15:23 part-00001
4 -rw-r--r-- 1 root root    8 Feb  3 15:23 part-00002
4 -rw-r--r-- 1 root root    7 Feb  3 15:23 part-00003
4 -rw-r--r-- 1 root root    9 Feb  3 15:23 part-00004
4 -rw-r--r-- 1 root root    9 Feb  3 15:23 part-00005
4 -rw-r--r-- 1 root root   11 Feb  3 15:23 part-00006
4 -rw-r--r-- 1 root root    9 Feb  3 15:23 part-00007
4 -rw-r--r-- 1 root root   12 Feb  3 15:23 part-00008
0 -rw-r--r-- 1 root root    0 Feb  3 15:23 part-00009


### Calculate Relative Frequency and Sort by TOP and BOTTOM

In [ ]:
%%writefile WordCount3.3.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re

WORD_RE = re.compile(r"[\w']+")


class MRWordCount33(MRJob):

    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_words,
                   combiner=self.combiner_count_words,
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_find_max_word)
        ]

    def mapper_get_words(self, _, line):
        for word in WORD_RE.findall(line):
            self.increment_counter('Process', 'Mapper', 1)
            yield (word.lower(), 1)

    def combiner_count_words(self, word, counts):
        # optimization: sum the words we've seen so far
        yield (word, sum(counts))

    def reducer_count_words(self, word, counts):
        # send all (num_occurrences, word) pairs to the same reducer.
        # num_occurrences is so we can easily use Python's max() function.
        yield None, (sum(counts), word)

    # discard the key; it is just None
    def reducer_find_max_word(self, _, word_count_pairs):
        # each item of word_count_pairs is (count, word),
        # so yielding one results in key=counts, value=word
        yield max(word_count_pairs)


if __name__ == '__main__':
    MRMostUsedWord.run()